# Multi-Agent Interview Coach

Мультиагентная система для проведения технического интервью: агенты Interviewer, Observer и Evaluator, скрытая рефлексия, диалог о проектах и технические вопросы, финальный структурированный фидбэк

In [20]:
pip install langchain langchain-openai langchain-anthropic langchain-mistralai langchain-google-genai langgraph python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [21]:
import os
import json
from typing import TypedDict, Annotated, Sequence
from datetime import datetime
import re
from langgraph.graph import StateGraph, END
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate


## 2. Выбор модели

<small>загрузка .env, выбор провайдера LLM и инициализация модели.</small>

In [ ]:
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass

LLM_PROVIDER = os.getenv("LLM_PROVIDER", "deepseek")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY", "")
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY", "")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", "")
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY", "")

def get_llm():
    if LLM_PROVIDER == "anthropic":
        from langchain_anthropic import ChatAnthropic
        return ChatAnthropic(model="claude-3-5-sonnet-20241022", api_key=ANTHROPIC_API_KEY, temperature=0.5)
    
    elif LLM_PROVIDER == "gigachat":
        from langchain_openai import ChatOpenAI
        return ChatOpenAI(model="GigaChat", base_url=os.getenv("GIGACHAT_BASE_URL", "https://gigachat.dev/api/v1"), api_key=os.getenv("GIGACHAT_API_KEY", OPENAI_API_KEY), temperature=0.5)
    
    elif LLM_PROVIDER == "mistral":
        from langchain_mistralai import ChatMistralAI
        model = os.getenv("MISTRAL_MODEL", "mistral-small-latest")
        return ChatMistralAI(model=model, api_key=MISTRAL_API_KEY, temperature=0.5)
    
    elif LLM_PROVIDER == "gemini":
        from langchain_google_genai import ChatGoogleGenerativeAI
        model = os.getenv("GOOGLE_MODEL", "gemini-1.5-flash")
        return ChatGoogleGenerativeAI(model=model, api_key=GOOGLE_API_KEY, temperature=0.5)
    
    elif LLM_PROVIDER == "deepseek":
        from langchain_openai import ChatOpenAI
        model = os.getenv("DEEPSEEK_MODEL", "deepseek-chat")
        return ChatOpenAI(model=model, base_url="https://api.deepseek.com", api_key='API', temperature=0.5)
    
    else:
        from langchain_openai import ChatOpenAI
        return ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.5)

llm = get_llm()
print(f"LLM: {LLM_PROVIDER}")

LLM: deepseek


## 2. Устойчивость к вбросам 

<small>класс проверки off-topic по ключевым словам и редирект-сообщение для возврата к интервью.</small>

In [23]:
class RobustnessValidator:
    OFF_TOPIC_KEYWORDS = [
        "погода", "weather", "политика", "politics", "спорт", "sport",
        "футбол", "football", "кино", "movie", "музыка", "music",
        "еда", "food", "путешествие", "travel"
    ]
    ON_TOPIC_KEYWORDS = [
        "код", "code", "программ", "program", "разработка", "develop",
        "язык", "language", "библиотека", "library", "фреймворк", "framework",
        "база данных", "database", "алгоритм", "algorithm", "тест", "test",
        "архитектура", "architecture", "проект", "project", "опыт", "experience",
        "python", "java", "sql", "api", "git"
    ]
    EVASION_PATTERNS = [
        r"давай(те)?\s+поговорим\s+о", r"а\s+можно\s+о", r"лучше\s+расскажи", r"смени\s+тему"
    ]

    @staticmethod
    def is_off_topic(message: str) -> bool:
        if not message or len(message.strip().split()) < 3:
            return False
        low = message.lower()
        off = sum(1 for k in RobustnessValidator.OFF_TOPIC_KEYWORDS if k in low)
        on = sum(1 for k in RobustnessValidator.ON_TOPIC_KEYWORDS if k in low)
        if off > 0 and on == 0:
            return True
        for p in RobustnessValidator.EVASION_PATTERNS:
            if re.search(p, low):
                return True
        return False

    @staticmethod
    def get_redirect_message() -> str:
        return (
            "Спасибо за ваш ответ. Давайте вернёмся к техническим вопросам интервью — "
            "так я смогу лучше оценить ваши навыки для позиции."
        )

## 3. Состояние графа 

<small>определение InterviewState (messages, turns, internal_thoughts, difficulty, topics_covered и др.) для графа LangGraph.</small>

In [24]:
class InterviewState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], "история сообщений"]
    participant_name: str
    position: str
    grade: str
    experience: str
    candidate_name: str
    internal_thoughts: Annotated[list[str], "логи внутренних мыслей агентов"]
    turns: Annotated[list[dict], "зафиксированные ходы для лога"]
    current_difficulty: str  
    topics_covered: Annotated[list[str], "темы, уже затронутые"]
    stop_requested: bool
    final_feedback: str
    off_topic_count: int 
    performance_history: Annotated[list[float], "оценки Evaluator по ходам 0..1"]

## 4. Промпты агентов

<small>системные промпты для Observer, Interviewer и Evaluator (формат THOUGHT/INSTRUCTION/DIFFICULTY/TOPICS и оценка 0–1).</small>

In [25]:
OBSERVER_SYSTEM = """Ты — Observer (Наблюдатель) в техническом интервью. Ты не общаешься с кандидатом напрямую.
Твоя задача:
1. Проанализировать последний ответ кандидата: точность фактов, уверенность, полнота.
2. Определить, не ушёл ли разговор off-topic (погода, личное и т.д.) или кандидат не выдал ли ложные факты (галлюцинации). В таких случаях предложи вежливо вернуть беседу в русло интервью.
3. Учитывая контекст (позиция: {position}, грейд: {grade}, опыт: {experience}) и уже затронутые темы {topics_covered}, решить:
   - какую тему задать дальше (не повторять вопросы, на которые уже был ответ);
   - повысить сложность (current_difficulty -> hard), оставить (medium) или понизить (easy), если кандидат "плывёт".
4. Фаза интервью: как только известны грейд и опыт кандидата (из первого сообщения) — сразу переходи к техническим вопросам по позиции (Python, БД, API и т.д.). Фазы «вопросы о проектах» нет: после представления кандидата — только технические вопросы. Всего около 8 сообщений от кандидата за интервью, за это время нужно проверить его.
5. Если кандидат ушёл от ответа, задав встречный вопрос (например, «а как у вас?», «а что вы имеете в виду?»): в INSTRUCTION укажи, что интервьюер должен кратко ответить на вопрос кандидата (1–2 предложения), а затем вежливо вернуть разговор к исходному вопросу и повторить или переформулировать его.
6. Выдать ответ СТРОГО в формате (все блоки обязательны):
   THOUGHT: [твои выводы и рекомендации на 1-3 предложения]
   INSTRUCTION: [что сказать/спросить дальше: конкретный вопрос или действие]
   DIFFICULTY: [easy | medium | hard] — следующая сложность вопросов
   TOPICS: [тема текущего/следующего вопроса, одна или через запятую]
Отвечай только на русском. Не придумывай факты."""

INTERVIEWER_SYSTEM = """Ты — Interviewer (Интервьюер) на техническом интервью. Ты ведёшь диалог с кандидатом.
Контекст: позиция {position}, грейд {grade}, опыт кандидата: {experience}. Имя кандидата: {candidate_name}.
Ты получил от Observer инструкцию (INSTRUCTION). Выполни её: задай один вопрос или сделай короткий комментарий, затем задай следующий вопрос.
Правила:
- После того как узнали грейд и опыт кандидата (из представления) — сразу переходи к техническим вопросам по позиции. Отдельной фазы про проекты нет.
- Говори от первого лица (интервьюер). Один блок реплики — без разметки, без "Interviewer:".
- Если инструкция говорит вернуть беседу в русло — вежливо переведи разговор обратно к теме интервью.
- Не задавай вопросов, на которые кандидат уже ответил в этой беседе.
- Адаптируй тон: при слабом ответе можно дать подсказку или упростить следующий вопрос.
- Если кандидат вместо ответа задал встречный вопрос (уклонился от ответа): сначала кратко (1–2 предложения) ответь на его вопрос, затем вежливо верни разговор к своему исходному вопросу и повтори или переформулируй его, чтобы получить ответ.
Отвечай только текстом реплики для кандидата, без префиксов."""

EVALUATOR_SYSTEM = """Ты — Evaluator (Оценщик) в техническом интервью. Ты не общаешься с кандидатом.
Оцени ответ кандидата на заданный вопрос: фактическая корректность (correct/partial/incorrect), полнота, глубина.
Дай числовую оценку 0.0–1.0. Если ответ с ошибками — укажи правильный ответ в блоке ПРАВИЛЬНЫЙ_ОТВЕТ:.
Формат ответа:
ОЦЕНКА: число 0.0–1.0
КОРРЕКТНОСТЬ: correct | partial | incorrect
КОММЕНТАРИЙ: 1–2 предложения
ПРАВИЛЬНЫЙ_ОТВЕТ: (если нужен — кратко)
Отвечай на русском."""

## 5. Скрытая рефлексия

<small>узел Observer: анализ последнего ответа кандидата, вызов LLM, парсинг DIFFICULTY/TOPICS и обновление internal_thoughts.</small>

In [26]:
def _format_history(messages: Sequence[BaseMessage], max_last: int = 12) -> str:
    parts = []
    for m in list(messages)[-max_last:]:
        if isinstance(m, HumanMessage):
            parts.append(f"Кандидат: {m.content}")
        else:
            parts.append(f"Интервьюер: {m.content}")
    return "\n".join(parts) if parts else "(пока нет реплик)"

def observer_node(state: InterviewState) -> dict:
    messages = state["messages"]
    last_user = None
    for m in reversed(messages):
        if isinstance(m, HumanMessage):
            last_user = m.content
            break
    if not last_user:
        last_user = "(приветствие, ожидаем первый ответ)"

    prompt = ChatPromptTemplate.from_messages([
        ("system", OBSERVER_SYSTEM),
        ("human", "История диалога:\n{history}\n\nПоследний ответ кандидата: {last_reply}\n\nГрейд и опыт уже известны: {intro_known}. Если да — задавай только технический вопрос по позиции; иначе можно уточнить представление. Текущая сложность: {difficulty}. Дай THOUGHT, INSTRUCTION, DIFFICULTY и TOPICS.")
    ])
    chain = prompt | llm
    intro_known = bool((state.get("grade") or "").strip() and (state.get("experience") or "").strip())
    response = chain.invoke({
        "position": state.get("position", "") or "(не указано)",
        "grade": state.get("grade", "") or "(не указано)",
        "experience": state.get("experience", "") or "(не указано)",
        "topics_covered": state.get("topics_covered") or [],
        "history": _format_history(messages),
        "last_reply": last_user,
        "intro_known": "да" if intro_known else "нет",
        "difficulty": state.get("current_difficulty", "medium"),
    })
    text = response.content if hasattr(response, "content") else str(response)
    thoughts = state.get("internal_thoughts") or []
    thoughts.append(f"[Observer]: {text}")
    out = {"internal_thoughts": thoughts}
    if "DIFFICULTY:" in text:
        d = text.split("DIFFICULTY:")[-1].strip().split()[0].lower()
        if d in ("easy", "medium", "hard"):
            out["current_difficulty"] = d
    if "TOPICS:" in text:
        raw = text.split("TOPICS:")[-1].strip().split("THOUGHT:")[0].strip().split("INSTRUCTION:")[0].strip()
        new_topics = [t.strip() for t in raw.split(",") if t.strip()]
        if new_topics:
            prev = list(state.get("topics_covered") or [])
            out["topics_covered"] = prev + [t for t in new_topics if t not in prev]
    return out

## 6. Evaluator (скрытый оценщик)

<small>узел Evaluator: оценка ответа кандидата 0–1 по вопросу интервьюера, дополнение performance_history и internal_thoughts.</small>

In [27]:
def evaluator_node(state: InterviewState) -> dict:
    messages = state.get("messages") or []
    last_user, last_question = "", ""
    for m in reversed(messages):
        if isinstance(m, HumanMessage):
            last_user = m.content or ""
            break
    for m in reversed(messages):
        if isinstance(m, AIMessage):
            last_question = m.content or ""
            break
    if not last_user:
        return {"performance_history": list(state.get("performance_history") or []), "internal_thoughts": state.get("internal_thoughts") or []}
    prompt = ChatPromptTemplate.from_messages([
        ("system", EVALUATOR_SYSTEM),
        ("human", "Вопрос интервьюера: {question}\n\nОтвет кандидата: {reply}\n\nДай ОЦЕНКА, КОРРЕКТНОСТЬ, КОММЕНТАРИЙ и при необходимости ПРАВИЛЬНЫЙ_ОТВЕТ.")
    ])
    chain = prompt | llm
    resp = chain.invoke({"question": last_question, "reply": last_user})
    text = resp.content if hasattr(resp, "content") else str(resp)
    score = 0.5
    m = re.search(r'ОЦЕНКА:\s*([0-9.]+)', text, re.I)
    if m:
        try:
            score = min(1.0, max(0.0, float(m.group(1))))
        except ValueError:
            pass
    hist = list(state.get("performance_history") or [])
    hist.append(score)
    thoughts = list(state.get("internal_thoughts") or [])
    thoughts.append(f"[Evaluator]: {text[:300]}..." if len(text) > 300 else f"[Evaluator]: {text}")
    return {"performance_history": hist, "internal_thoughts": thoughts}

## 7. Interviewer (видимая реплика)

<small>узел Interviewer: извлечение INSTRUCTION из Observer, вызов LLM и добавление одной реплики в messages.</small>

In [28]:
def _extract_instruction(observer_text: str) -> str:
    if "INSTRUCTION:" in observer_text:
        block = observer_text.split("INSTRUCTION:")[-1].strip()
        if "DIFFICULTY:" in block:
            block = block.split("DIFFICULTY:")[0].strip()
        if "THOUGHT:" in block:
            block = block.split("THOUGHT:")[0].strip()
        return block
    return observer_text

def interviewer_node(state: InterviewState) -> dict:
    thoughts = state.get("internal_thoughts") or []
    last_thought = thoughts[-1] if thoughts else ""
    instruction = _extract_instruction(last_thought)

    prompt = ChatPromptTemplate.from_messages([
        ("system", INTERVIEWER_SYSTEM),
        ("human", "Инструкция от Observer: {instruction}\n\nИстория диалога (последние реплики):\n{history}\n\nНапиши только одну реплику интервьюера для кандидата.")
    ])
    chain = prompt | llm
    response = chain.invoke({
        "position": state["position"],
        "grade": state["grade"],
        "experience": state["experience"],
        "participant_name": state.get("participant_name", ""),
        "candidate_name": state.get("candidate_name") or "Кандидат",
        "instruction": instruction,
        "history": _format_history(state["messages"]),
    })
    reply = response.content if hasattr(response, "content") else str(response)
    reply = reply.strip().strip('"')
    new_messages = list(state["messages"]) + [AIMessage(content=reply)]
    return {"messages": new_messages}

## 8. Условие: стоп или продолжение

<small>STOP_PHRASES, should_stop, should_redirect, off_topic_check_node и redirect_save_node для ветвления графа.</small>

In [29]:
STOP_PHRASES = ["стоп интервью", "завершить интервью", "закончить", "стоп", "конец интервью", "finish"]

def should_stop(state: InterviewState) -> str:
    if state.get("stop_requested"):
        return "finish"
    messages = state.get("messages") or []
    for m in reversed(messages):
        if isinstance(m, HumanMessage):
            lower = (m.content or "").strip().lower()
            if any(p in lower for p in STOP_PHRASES):
                return "finish"
            break
    return "continue"

def should_redirect(state: InterviewState) -> str:
    if state.get("off_topic_count", 0) >= 2:
        return "redirect"
    return "continue"

def off_topic_check_node(state: InterviewState) -> dict:
    last_user = ""
    for m in reversed(state.get("messages") or []):
        if isinstance(m, HumanMessage):
            last_user = (m.content or "").strip()
            break
    count = state.get("off_topic_count", 0)
    if last_user and RobustnessValidator.is_off_topic(last_user):
        count += 1
    return {"off_topic_count": count}

def redirect_save_node(state: InterviewState) -> dict:
    msg = RobustnessValidator.get_redirect_message()
    turns = list(state.get("turns") or [])
    thoughts = list(state.get("internal_thoughts") or [])
    thoughts.append("[Observer]: Off-topic. Redirecting.")
    user_msg = ""
    for m in reversed(state.get("messages") or []):
        if isinstance(m, HumanMessage):
            user_msg = m.content or ""
            break
    turns.append({
        "turn_id": len(turns) + 1,
        "agent_visible_message": msg,
        "user_message": user_msg,
        "internal_thoughts": "[Observer]: Off-topic.\n[Interviewer]: Redirecting.\n",
        "performance_metrics": {"score": 0.0},
    })
    new_messages = list(state.get("messages") or []) + [AIMessage(content=msg)]
    return {"messages": new_messages, "internal_thoughts": thoughts, "turns": turns}

redirect_save_node

<function __main__.redirect_save_node(state: __main__.InterviewState) -> dict>

## 9. Регистрация хода в лог (turn)

<small>save_turn_node и log_stop_turn_node: форматирование internal_thoughts для лога и добавление хода в turns.</small>

In [30]:
def _extract_thought(observer_text: str) -> str:
    if "THOUGHT:" in observer_text:
        block = observer_text.split("THOUGHT:")[1].strip()
        if "INSTRUCTION:" in block:
            block = block.split("INSTRUCTION:")[0].strip()
        return block[:500] if len(block) > 500 else block
    return observer_text[:500] if observer_text else ""

def _extract_evaluator_thought(text: str) -> str:
    if "[Evaluator]:" in text:
        return text.split("[Evaluator]:")[-1].strip()[:400]
    return text[:400] if text else ""

def _build_internal_thoughts_for_log(thoughts: list, agent_msg: str) -> str:
    """Формат: каждое сообщение [agent_name]: thought\n (по спецификации)."""
    parts = []
    if len(thoughts) >= 2:
        parts.append("[Observer]: " + _extract_thought(thoughts[-2]) + "\n")
    elif thoughts:
        parts.append("[Observer]: " + _extract_thought(thoughts[-1]) + "\n")
    if thoughts and "[Evaluator]" in (thoughts[-1] or ""):
        parts.append("[Evaluator]: " + _extract_evaluator_thought(thoughts[-1]) + "\n")
    interviewer_part = (agent_msg.split(".")[0].strip() + ".") if agent_msg else "Задал следующий вопрос."
    parts.append("[Interviewer]: " + interviewer_part + "\n")
    return "".join(parts)

def save_turn_node(state: InterviewState) -> dict:
    messages = list(state["messages"])
    turns = list(state.get("turns") or [])
    thoughts = state.get("internal_thoughts") or []
    user_msg = ""
    agent_msg = ""
    last_ai_before_user = ""
    for m in messages:
        if isinstance(m, HumanMessage):
            user_msg = m.content or ""
            agent_msg = last_ai_before_user
        else:
            last_ai_before_user = m.content or ""
    if not agent_msg:
        agent_msg = last_ai_before_user
    new_agent_msg = (messages[-1].content if messages and isinstance(messages[-1], AIMessage) else "") or last_ai_before_user
    turn_id = len(turns) + 1
    internal_str = _build_internal_thoughts_for_log(thoughts, new_agent_msg)
    perf = state.get("performance_history") or []
    score = perf[-1] if perf else 0.5
    turns.append({
        "turn_id": turn_id,
        "agent_visible_message": agent_msg,
        "user_message": user_msg,
        "internal_thoughts": internal_str,
        "performance_metrics": {"score": score},
    })
    return {"turns": turns}

def log_stop_turn_node(state: InterviewState) -> dict:
    turns = list(state.get("turns") or [])
    last_user = ""
    for m in reversed(state.get("messages") or []):
        if isinstance(m, HumanMessage):
            last_user = (m.content or "").strip()
            break
    if not last_user:
        return {}
    last_lower = last_user.lower()
    is_stop = any(p in last_lower for p in STOP_PHRASES)
    already_logged = turns and (turns[-1].get("user_message") or "").strip() == last_user
    if is_stop and not already_logged:
        turn_id = len(turns) + 1
        turns.append({
            "turn_id": turn_id,
            "agent_visible_message": "Интервью завершено по запросу. Формирую отчёт.",
            "user_message": last_user,
            "internal_thoughts": "[Observer]: Запрос на завершение.\n[Interviewer]: Завершение интервью.\n",
            "performance_metrics": {"score": 0.0},
        })
        return {"turns": turns}
    return {}


## 10. Генерация финального фидбэка

<small>feedback_node: вызов LLM с FEEDBACK_SYSTEM, парсинг JSON из ответа и формирование структурированного отчёта.</small>

In [31]:
FEEDBACK_SYSTEM = """Ты — эксперт по подведению итогов технического интервью. По истории диалога сформируй структурированный отчёт.
Кандидат (имя из представления): {candidate_name}. Позиция: {position}, грейд: {grade}, опыт: {experience}.
Сформируй отчёт строго в следующей структуре (на русском):

## Вердикт (Decision)
- **Grade:** Junior | Middle | Senior (выбери один)
- **Hiring Recommendation:** Hire | No Hire | Strong Hire
- **Confidence Score:** число 0-100 (насколько уверен в оценке)

## Анализ Hard Skills (Technical Review)
- **Confirmed Skills:** темы, где кандидат дал точные ответы (список).
- **Knowledge Gaps:** темы с ошибками или "не знаю"; для каждой приведи кратко правильный ответ.

## Soft Skills & Communication
- **Clarity:** насколько понятно излагал мысли.
- **Honesty:** честность (признание незнания vs попытки выкрутиться).
- **Engagement:** вовлечённость, встречные вопросы.

## Персональный Roadmap (Next Steps)
- Конкретные темы/технологии для подтягивания (список).
- Опционально: ссылки на документацию или статьи по темам.

Используй только факты из диалога. Не выдумывай.
При завершении интервью по запросу кандидата фидбэк должен быть осознанным: кратко опиши ответы кандидата по каждому вопросу (что спросили, что ответил, насколько полно и корректно), сделай выводы по ним и сформулируй вердикт на основе этих описаний.
Опционально в конце можно добавить JSON: {"grade": "...", "hiring_recommendation": "...", "confidence_score": N, "knowledge_gaps": [{"topic": "...", "user_answer": "...", "correct_answer": "..."}], "confirmed_skills": [], "roadmap": []}."""

def _parse_feedback_json(response_text: str) -> dict | None:
    try:
        start = response_text.find("{")
        end = response_text.rfind("}") + 1
        if start != -1 and end > start:
            return json.loads(response_text[start:end])
    except json.JSONDecodeError:
        pass
    return None

def _feedback_from_parsed(data: dict) -> str:
    parts = []
    parts.append("## Вердикт (Decision)")
    parts.append(f"- **Grade:** {data.get('grade', 'N/A')}")
    parts.append(f"- **Hiring Recommendation:** {data.get('hiring_recommendation', 'N/A')}")
    parts.append(f"- **Confidence Score:** {data.get('confidence_score', 0)}")
    parts.append("")
    parts.append("## Анализ Hard Skills")
    parts.append("- **Confirmed Skills:** " + ", ".join(data.get("confirmed_skills", [])))
    gaps = data.get("knowledge_gaps", [])
    if gaps:
        parts.append("- **Knowledge Gaps:**")
        for g in gaps:
            if isinstance(g, dict):
                parts.append(f"  - {g.get('topic', '')}: правильный ответ — {g.get('correct_answer', '')[:200]}")
    parts.append("")
    parts.append("## Roadmap")
    parts.append("\n".join("- " + x for x in data.get("roadmap", [])))
    return "\n".join(parts)

def _build_fallback_feedback(state: InterviewState) -> str:
    """Формирует осознанный структурированный фидбэк из состояния без вызова LLM: описание ответов кандидата и выводы."""
    parts = []
    parts.append("## Контекст кандидата")
    parts.append(f"- **Кандидат:** {state.get('candidate_name') or '—'}")
    parts.append(f"- **Позиция:** {state.get('position') or '—'}; **грейд:** {state.get('grade') or '—'}")
    parts.append(f"- **Опыт:** {(state.get('experience') or '—')[:400]}")
    turns = state.get("turns") or []
    perf = state.get("performance_history") or []
    parts.append("")
    parts.append("## Описание ответов кандидата по ходам интервью")
    for i, t in enumerate(turns):
        tid = t.get("turn_id", i + 1)
        q = (t.get("agent_visible_message") or "—").strip()
        a = (t.get("user_message") or "—").strip()
        sc = t.get("performance_metrics", {}).get("score")
        sc_str = f"{sc:.2f}" if isinstance(sc, (int, float)) else "—"
        parts.append(f"### Ход {tid}")
        parts.append(f"- **Вопрос:** {q}")
        parts.append(f"- **Ответ кандидата:** {a}")
        parts.append(f"- **Оценка ответа (0–1):** {sc_str}")
        parts.append("")
    parts.append("## Краткие выводы по ответам")
    if perf:
        avg = sum(perf) / len(perf)
        strong = sum(1 for p in perf if p >= 0.7)
        weak = sum(1 for p in perf if p < 0.5)
        parts.append(f"- Всего ходов с оценкой: {len(perf)}; средняя оценка: {avg:.2f}.")
        parts.append(f"- Ответов с оценкой ≥ 0.7: {strong}; с оценкой < 0.5: {weak}.")
    topics = state.get("topics_covered") or []
    if topics:
        parts.append(f"- Затронутые темы: {', '.join(topics)}.")
    parts.append("")
    parts.append("## Вердикт (Decision)")
    parts.append("- **Grade:** по данным диалога — см. описание ответов выше")
    parts.append("- **Hiring Recommendation:** требуется ручная оценка по описанию ответов кандидата")
    parts.append("- **Confidence Score:** 0 (отчёт сформирован по данным без LLM)")
    parts.append("")
    parts.append("## Roadmap")
    parts.append("- Рекомендуется оценить пробелы по описанию ответов выше и сформировать план развития.")
    return "\n".join(parts)

def feedback_node(state: InterviewState) -> dict:
    messages = state.get("messages") or []
    history = _format_history(messages, max_last=50) if messages else "(диалог пуст)"
    prompt = ChatPromptTemplate.from_messages([
        ("system", FEEDBACK_SYSTEM),
        ("human", "История диалога:\n{history}")
    ])
    chain = prompt | llm
    try:
        response = chain.invoke({
            "candidate_name": state.get("candidate_name") or "Кандидат",
            "position": state.get("position", "") or "(не указано)",
            "grade": state.get("grade", "") or "(не указано)",
            "experience": state.get("experience", "") or "(не указано)",
            "history": history,
        })
        text = response.content if hasattr(response, "content") else str(response)
        parsed = _parse_feedback_json(text)
        if parsed:
            text = _feedback_from_parsed(parsed)
    except Exception:
        text = _build_fallback_feedback(state)
    return {"final_feedback": text}

## 11. Сборка графа LangGraph

<small>build_graph: создание StateGraph с узлами и условными рёбрами (router → off_topic_check → observer → evaluator → interviewer → save_turn и т.д.).</small>

In [32]:
def router_node(state: InterviewState) -> dict:
    return {}

def build_graph():
    graph = StateGraph(InterviewState)
    graph.add_node("router", router_node)
    graph.add_node("observer", observer_node)
    graph.add_node("evaluator", evaluator_node)
    graph.add_node("interviewer", interviewer_node)
    graph.add_node("save_turn", save_turn_node)
    graph.add_node("log_stop_turn", log_stop_turn_node)
    graph.add_node("feedback", feedback_node)

    graph.add_node("off_topic_check", off_topic_check_node)
    graph.add_node("redirect_save", redirect_save_node)
    graph.set_entry_point("router")
    graph.add_conditional_edges("router", should_stop, {"continue": "off_topic_check", "finish": "log_stop_turn"})
    graph.add_conditional_edges("off_topic_check", should_redirect, {"redirect": "redirect_save", "continue": "observer"})
    graph.add_edge("redirect_save", END)
    graph.add_edge("observer", "evaluator")
    graph.add_edge("evaluator", "interviewer")
    graph.add_edge("interviewer", "save_turn")
    graph.add_conditional_edges("save_turn", should_stop, {"continue": END, "finish": "log_stop_turn"})
    graph.add_edge("log_stop_turn", "feedback")
    graph.add_edge("feedback", END)

    return graph.compile()

interview_graph = build_graph()
print("Graph built.")

Graph built.


## 12. Старт интервью (вводные)

<small>start_interview() возвращает начальный state с приветствием; parse_intro_from_message извлекает позицию, грейд и опыт из первого сообщения.</small>

In [33]:
TESTER_NAME = "Григорьев Владимир Сергеевич"

def start_interview() -> InterviewState:
    """Инициализация без параметров кандидата: имя, роль, грейд, опыт вводит сам человек в первом сообщении."""
    initial_msg = (
        "Здравствуйте! Представьтесь, пожалуйста: как вас зовут, на какую позицию и грейд претендуете, кратко об опыте. "
        "Тогда начнём интервью."
    )
    return {
        "messages": [AIMessage(content=initial_msg)],
        "participant_name": TESTER_NAME,
        "position": "",
        "grade": "",
        "experience": "",
        "candidate_name": "",
        "internal_thoughts": ["[Observer]: Старт интервью. [Interviewer]: Запрос представления."],
        "turns": [],
        "current_difficulty": "medium",
        "topics_covered": [],
        "stop_requested": False,
        "final_feedback": "",
        "off_topic_count": 0,
        "performance_history": [],
    }

def parse_intro_from_message(text: str) -> dict:
    """Из первого сообщения кандидата извлекаем позицию, грейд, опыт (и имя при возможности)."""
    t = (text or "").strip()
    out = {"position": "", "grade": "", "experience": t[:500], "candidate_name": ""}
    low = t.lower()
    for g in ("junior", "middle", "senior", "джуниор", "мидл", "сеньор", "младший", "старший"):
        if g in low:
            out["grade"] = g
            break
    for p in ("python", "backend", "frontend", "разработчик", "developer", "инженер", "engineer"):
        if p in low:
            out["position"] = t[:200] if len(t) < 200 else (t[:100] + "...")
            break
    if not out["position"]:
        out["position"] = t[:150] if len(t) < 150 else (t[:80] + "...")
    return out

## 13. Один шаг диалога (вопрос — ответ)

<small>step_interview добавляет сообщение пользователя, вызывает граф и при первом ответе заполняет контекст кандидата; get_last_agent_message и get_last_internal_thoughts для вывода.</small>

In [34]:
def _is_stop_phrase(msg: str) -> bool:
    lower = (msg or "").strip().lower()
    return any(p in lower for p in ["стоп интервью", "завершить", "конец интервью", "давай фидбэк", "закончить", "стоп", "finish"])

def _ensure_feedback_on_stop(state: InterviewState) -> InterviewState:
    """При стопе всегда формируем фидбэк: log_stop_turn + feedback_node."""
    log_update = log_stop_turn_node(state)
    state = {**state, **log_update}
    try:
        feedback_update = feedback_node(state)
        return {**state, **feedback_update}
    except Exception:
        return {**state, "final_feedback": _build_fallback_feedback(state)}

def step_interview(current_state: InterviewState, user_message: str) -> InterviewState:
    new_messages = list(current_state.get("messages") or []) + [HumanMessage(content=user_message)]
    new_state = {**current_state, "messages": new_messages}
    # Сразу заполняем грейд/опыт из сообщения, чтобы Observer при первом же ответе переходил к техническим вопросам
    if user_message.strip():
        parsed = parse_intro_from_message(user_message)
        for k in ("position", "grade", "experience", "candidate_name"):
            if parsed.get(k) and not (new_state.get(k) or "").strip():
                new_state[k] = parsed.get(k) or new_state.get(k) or ""
    result = None
    try:
        result = interview_graph.invoke(new_state)
    except Exception as e:
        if _is_stop_phrase(user_message):
            result = _ensure_feedback_on_stop(new_state)
        else:
            raise
    # Гарантия: если пользователь сказал стоп, но фидбэка нет — формируем вручную
    if _is_stop_phrase(user_message) and not (result.get("final_feedback") or "").strip():
        result = _ensure_feedback_on_stop(result if result else new_state)
    if not (result.get("position") or result.get("experience")) and user_message.strip():
        parsed = parse_intro_from_message(user_message)
        result = {**result, "position": parsed.get("position") or result.get("position", ""), "grade": parsed.get("grade") or result.get("grade", ""), "experience": parsed.get("experience") or result.get("experience", ""), "candidate_name": parsed.get("candidate_name") or result.get("candidate_name", "")}
    return result

def get_last_agent_message(state: InterviewState) -> str:
    for m in reversed(state.get("messages") or []):
        if isinstance(m, AIMessage):
            return m.content or ""
    return ""

def get_last_internal_thoughts(state: InterviewState, last_n: int = 2) -> list:
    thoughts = state.get("internal_thoughts") or []
    return thoughts[-last_n:] if len(thoughts) >= last_n else thoughts

## 14. Цикл интервью 

<small>run_interview_loop: интерактивный цикл с input, вывод реплики интервьюера и internal_thoughts до появления final_feedback.</small>

In [35]:
def run_interview_loop(max_turns: int = 15):
    state = start_interview()
    print("--- Приветствие ---")
    print(get_last_agent_message(state))
    print()

    for _ in range(max_turns - 1):
        user_input = input("Вы (кандидат): ").strip()
        if not user_input:
            continue
        state = step_interview(state, user_input)
        if state.get("final_feedback"):
            break
        print("--- Интервьюер ---")
        print(get_last_agent_message(state))
        print("--- Внутренние мысли (логи) ---")
        for t in get_last_internal_thoughts(state):
            print(t[:300] + "..." if len(t) > 300 else t)
        print()

    if state.get("final_feedback"):
        print("\n=== Финальный фидбэк ===")
        print(state["final_feedback"])
    return state

## 14. Запуск интервью (пример без input — для автопроверки)

<small>автозапуск с тестовыми ответами test_replies без запроса ввода для быстрой проверки пайплайна.</small>

In [36]:
state = start_interview()
print("Приветствие:", get_last_agent_message(state), "\n")

test_replies = [
    "Я джун, пишу код 3 месяца на Python.",
    "Знаю списки, словари, функции. Декораторы использовал в FastAPI.",
    "GIL — это глобальная блокировка в CPython, один поток выполняет байткод в момент времени.",
]
for reply in test_replies:
    print("Кандидат:", reply)
    state = step_interview(state, reply)
    if state.get("final_feedback"):
        break
    print("Интервьюер:", get_last_agent_message(state))
    print("Мысли:", get_last_internal_thoughts(state))
    print()

if state.get("final_feedback"):
    print("=== Финальный фидбэк ===")
    print(state["final_feedback"])

Приветствие: Здравствуйте! Представьтесь, пожалуйста: как вас зовут, на какую позицию и грейд претендуете, кратко об опыте. Тогда начнём интервью. 

Кандидат: Я джун, пишу код 3 месяца на Python.
Интервьюер: Хорошо, спасибо. Тогда сразу перейдём к техническим вопросам по Python. Расскажите, пожалуйста, чем отличаются списки (list) от кортежей (tuple) в Python?
Мысли: ['[Observer]: THOUGHT: Кандидат представился кратко: указал грейд (джун) и опыт (3 месяца на Python), но не назвал имя и позицию. Однако для начала интервью этого достаточно — грейд и опыт известны, можно сразу переходить к техническим вопросам по позиции (Python-разработчик). Сложность оставим medium, так как кандидат — джун, но нужно проверить базовые знания.\n\nINSTRUCTION: Отлично, спасибо за представление. Тогда начнём с технических вопросов по Python. Расскажите, в чём разница между списком (list) и кортежем (tuple) в Python? Приведите примеры использования каждого.\n\nDIFFICULTY: medium  \nTOPICS: Python, структуры 

## 15. Сохранение лога в interview_log.json

<small>save_interview_log записывает participant_name, turns и final_feedback в JSON; test_logger прогоняет сценарий и сохраняет в interview_log_{N}.json.</small>

In [37]:
def save_interview_log(state: InterviewState, filepath: str | None = None, strict_format: bool = False) -> str:
    if filepath is None or filepath == "":
        filepath = "interview_log.json"
    turns_raw = state.get("turns") or []
    if strict_format:
        turns = [
            {"turn_id": t.get("turn_id"), "agent_visible_message": t.get("agent_visible_message", ""), "user_message": t.get("user_message", ""), "internal_thoughts": t.get("internal_thoughts", "")}
            for t in turns_raw
        ]
    else:
        turns = turns_raw
    log = {
        "participant_name": state.get("participant_name", TESTER_NAME),
        "turns": turns,
        "final_feedback": state.get("final_feedback") or "",
    }
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(log, f, ensure_ascii=False, indent=2)
    return filepath


# saved_path = save_interview_log(state)
# print(f"Сохранено: {saved_path}")

## Запуск:

<small>интерактивный запуск run_interview_loop и сохранение лога в interview_log.json после завершения интервью.</small>

In [40]:
state = run_interview_loop(max_turns=15)
save_interview_log(state)  

--- Приветствие ---
Здравствуйте! Представьтесь, пожалуйста: как вас зовут, на какую позицию и грейд претендуете, кратко об опыте. Тогда начнём интервью.

--- Интервьюер ---
Спасибо, Алексей. Давайте перейдём к технической части. Расскажите, как бы вы подошли к задаче детекции объектов на видео в реальном времени, если есть требования и к точности, и к скорости обработки?
--- Внутренние мысли (логи) ---
[Observer]: THOUGHT: Кандидат представился кратко, но достаточно: указал имя, позицию и грейд. Опыт не описан, но для начала интервью этого достаточно — можно сразу переходить к техническим вопросам, как и требуется. Начинаем с темы компьютерного зрения, сложность оставляем medium, так как это первы...
[Evaluator]: ОЦЕНКА: 0.2
КОРРЕКТНОСТЬ: partial
КОММЕНТАРИЙ: Кандидат назвал имя и общую область позиции, но не указал грейд (Senior упомянут, но не в структурированном виде), компанию (если она была в описании вакансии) и не дал краткого описания опыта, как требовалось в вопросе.
ПРАВИЛЬН

'interview_log.json'

## test_logger() — для финального тестирования

<small>прогон сценария по списку ответов пользователя и сохранение лога в interview_log_{номер_сценария}.json.</small>

In [39]:
def test_logger(scenario_number: int, user_replies: list[str]) -> str:
    state = start_interview()
    for reply in user_replies:
        state = step_interview(state, reply)
        if state.get("final_feedback"):
            break
    filepath = f"interview_log_{scenario_number}.json"
    save_interview_log(state, filepath, strict_format=True)
    return filepath

filepath = test_logger(
    scenario_number=1,
    user_replies=[
        "Меня зовут Алексей. Претендую на Python-разработчик, Middle. Около 3 лет опыта: Django, FastAPI, PostgreSQL.",
        "List — изменяемый, tuple — неизменяемый. List для динамических коллекций, tuple для ключей и констант.",
        "Стоп интервью. Давай фидбэк.",
    ],
)
print(f"Сохранено: {filepath}")

Сохранено: interview_log_1.json
